## Part 1

In [210]:
from collections import defaultdict

In [334]:
TEST_INFILE = "inputs/day_22_test_1.txt"
INFILE = "inputs/day_22_1.txt"

#with open(TEST_INFILE) as infile:
with open(INFILE) as infile:
    lines = infile.read().splitlines()
bricks = [[eval(e) for e in line.split("~")] for line in lines]

In [335]:
len(bricks)

1257

In [336]:
test_ids_map = {
    0: "A",
    1: "B",
    2: "C",
    3: "D",
    4: "E",
    5: "F",
    6: "G",
}

In [337]:
def compare_ends(brick):
    return len(list(filter(lambda e: not e, [e1==e2 for e1, e2, in zip(brick[0], brick[1])])))

In [338]:
# verify that all bricks only differ on a single dimension (or are a single cube)
list(filter(lambda e: e not in [0, 1], [compare_ends(b) for b in bricks]))

[]

In [339]:
min_x = min([min(b[0][0], b[1][0]) for b in bricks])
max_x = max([max(b[0][0], b[1][0]) for b in bricks])
min_y = min([min(b[0][1], b[1][1]) for b in bricks])
max_y = max([max(b[0][1], b[1][1]) for b in bricks])
(min_x, max_x), (min_y, max_y)

((0, 9), (0, 9))

In [340]:
from enum import Enum

class XYPoint:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"({self.x}, {self.y})"

    def __add__(self, other):
        return XYPoint(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return XYPoint(self.x - other.x, self.y - other.y)

    def __mul__(self, number):
        return XYPoint(self.x * number, self.y * number)

    def __radd__(self, other):
        return self + other
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __hash__(self):
        return hash((self.x, self.y))


assert XYPoint(0, 0) == XYPoint(0, 0)
assert XYPoint(1, 0) + XYPoint(1, 10) == XYPoint(2, 10)
p = XYPoint(1, 0)
p += XYPoint(1, 10)
assert p == XYPoint(2, 10)
assert XYPoint(1, 2) * 10 == XYPoint(10, 20)
assert XYPoint(20, 10) - XYPoint(5, 5) == XYPoint(15, 5)


class XYLineSegment:
    def __init__(self, start: XYPoint, end: XYPoint):
        self.start = start
        self.end = end
        if start == end:
            self.orientation = "point"
        elif start.y == end.y:
            self.orientation = "x"
        elif start.x == end.x:
            self.orientation = "y"
        else:
            raise ValueError("Something is wrong!!")
        
    def __repr__(self):
        return f"[{self.start} -> {self.end}, {self.orientation}]"
    
    def __len__(self):
        if self.orientation == "point":
            return 1
        elif self.orientation == "x":
            return abs(self.end.x - self.start.x) + 1
        elif self.orientation == "y":
            return abs(self.end.y - self.start.y) + 1
        else:
            raise ValueError("Something is wrong!!")


    def intersects(self, other):
        self_left   = min(self.start.x, self.end.x)
        self_right  = max(self.start.x, self.end.x)
        other_left  = min(other.start.x, other.end.x)
        other_right = max(other.start.x, other.end.x)
        
        self_top     = max(self.start.y, self.end.y)
        self_bottom  = min(self.start.y, self.end.y)
        other_top    = max(other.start.y, other.end.y)
        other_bottom = min(other.start.y, other.end.y)
        
        # both are points
        if self.orientation == "point" and other.orientation == "point":
            return self.start == other.start
        
        # one is a point and the other is a line
        elif self.orientation == "point" and other.orientation == "x":
            return self.start.y == other.start.y and (other_left <= self.start.x <= other_right)
        elif self.orientation == "point" and other.orientation == "y":
            return self.start.x == other.start.x and (other_bottom <= self.start.y <= other_top)
        elif self.orientation == "x" and other.orientation == "point":
            return other.intersects(self)
        elif self.orientation == "y" and other.orientation == "point":
            return other.intersects(self)
        
        # both are horizontal lines
        elif self.orientation == other.orientation and self.orientation == "x":
            # vertically offset
            if self.start.y != other.start.y:
                return False
            # overlap
            elif (self_left <= other_left <= self_right) or (self_left <= other_right <= self_right) or \
                (other_left <= self_left <= other_right) or (other_left <= self_right <= other_right):
                return True
            else:
                return False

        # both are vertical lines
        elif self.orientation == other.orientation and self.orientation == "y":
            # horizontally offset
            if self.start.x != other.start.x:
                return False
            # overlap
            elif (self_bottom <= other_bottom <= self_top) or (self_bottom <= other_top <= self_top) or \
                (other_bottom <= self_bottom <= other_top) or (other_bottom <= self_top <= other_top):
                return True
            else:
                return False

        # self is horizontal, other is vertical
        elif self.orientation != other.orientation and self.orientation == "x":
            if other_bottom <= self.start.y <= other_top and (self_left <= other_left <= self_right or self_left <= other_right <= self_right):
                return True
            return False
        # self is vertical, other is horizontal
        elif self.orientation != other.orientation and self.orientation == "y":
            if other_left <= self.start.x <= other_right and (self_bottom <= other_bottom <= self_top or self_bottom <= other_top <= self_top):
                return True
            return False


assert len(XYLineSegment(XYPoint(0, 0), XYPoint(0, 0))) == 1
assert len(XYLineSegment(XYPoint(0, 0), XYPoint(0, 2))) == 3
assert XYLineSegment(XYPoint(0, 0), XYPoint(0, 0)).orientation == "point"
assert XYLineSegment(XYPoint(0, 0), XYPoint(2, 0)).orientation == "x"
assert XYLineSegment(XYPoint(0, 0), XYPoint(0, 2)).orientation == "y"

# vertical line
test_l = XYLineSegment(XYPoint(0, 0), XYPoint(0, 10))
# intersecting point
t = XYLineSegment(XYPoint(0, 5), XYPoint(0, 5))
assert test_l.intersects(t) == True
assert t.intersects(test_l) == True
# overlapping vertical
t = XYLineSegment(XYPoint(0, 2), XYPoint(0, 5))
assert test_l.intersects(t) == True
assert t.intersects(test_l) == True
t = XYLineSegment(XYPoint(0, 5), XYPoint(0, 2))
assert test_l.intersects(t) == True
assert t.intersects(test_l) == True
# crossing horizontal
t = XYLineSegment(XYPoint(-10, 2), XYPoint(5, 2))
assert test_l.intersects(t) == True
assert t.intersects(test_l) == True
# crossing horizontal just touching at edge
t = XYLineSegment(XYPoint(0, 2), XYPoint(5, 2))
assert test_l.intersects(t) == True
assert t.intersects(test_l) == True

# horizontal line
test_l = XYLineSegment(XYPoint(0, 0), XYPoint(10, 0))
# intersecting point
t = XYLineSegment(XYPoint(5, 0), XYPoint(5, 0))
assert test_l.intersects(t) == True
assert t.intersects(test_l) == True
# overlapping horizontal
t = XYLineSegment(XYPoint(2, 0), XYPoint(5, 0))
assert test_l.intersects(t) == True
assert t.intersects(test_l) == True
t = XYLineSegment(XYPoint(5, 0), XYPoint(2, 0))
assert test_l.intersects(t) == True
assert t.intersects(test_l) == True
# crossing vertical
t = XYLineSegment(XYPoint(2, -10), XYPoint(2, 5))
assert test_l.intersects(t) == True
assert t.intersects(test_l) == True
# crossing vertical just touching at edge
t = XYLineSegment(XYPoint(2, 0), XYPoint(2, 5))
assert test_l.intersects(t) == True
assert t.intersects(test_l) == True

# vertical line
test_l = XYLineSegment(XYPoint(0, 0), XYPoint(0, 10))
# non-intersecting point
t = XYLineSegment(XYPoint(0, 20), XYPoint(0, 20))
assert t.intersects(test_l) == False
assert test_l.intersects(t) == False
# non-intersecting vertical
t = XYLineSegment(XYPoint(0, 20), XYPoint(0, 30))
assert t.intersects(test_l) == False
assert test_l.intersects(t) == False
# non-intersecting horizontal
t = XYLineSegment(XYPoint(0, 20), XYPoint(10, 20))
assert test_l.intersects(t) == False
assert t.intersects(test_l) == False

# horizontal line
test_l = XYLineSegment(XYPoint(0, 0), XYPoint(10, 0))
# non-intersecting point
t = XYLineSegment(XYPoint(20, 0), XYPoint(20, 0))
assert test_l.intersects(t) == False
assert t.intersects(test_l) == False
# non-intersecting horizontal
t = XYLineSegment(XYPoint(0, 20), XYPoint(10, 20))
assert t.intersects(test_l) == False
assert test_l.intersects(t) == False
# non-intersecting vertical
t = XYLineSegment(XYPoint(20, 0), XYPoint(20, 10))
assert t.intersects(test_l) == False
assert test_l.intersects(t) == False


class Brick:
    def __init__(self, id: int, end_1: tuple, end_2: tuple):
        self.id = id
        self.end_1 = end_1
        self.end_2 = end_2
        self.line = XYLineSegment(XYPoint(end_1[0], end_1[1]), XYPoint(end_2[0], end_2[1]))
        self.z    = list(range(min(end_1[2], end_2[2]), max(end_1[2], end_2[2]) + 1))

    def __repr__(self):
        return f"Brick #{self.id}: [{self.end_1}, {self.end_2}] => {self.line}, z={self.z}"



In [341]:
#bricks = [Brick(test_ids_map[brick_num], brick[0], brick[1]) for brick_num, brick in enumerate(bricks)]
bricks = [Brick(brick_num, brick[0], brick[1]) for brick_num, brick in enumerate(bricks)]
bricks = sorted(bricks, key=lambda b: -b.z[-1])

In [342]:
bricks

[Brick #101: [(4, 5, 341), (7, 5, 341)] => [(4, 5) -> (7, 5), x], z=[341],
 Brick #43: [(7, 5, 339), (9, 5, 339)] => [(7, 5) -> (9, 5), x], z=[339],
 Brick #826: [(4, 1, 337), (4, 1, 339)] => [(4, 1) -> (4, 1), point], z=[337, 338, 339],
 Brick #889: [(6, 0, 338), (6, 1, 338)] => [(6, 0) -> (6, 1), y], z=[338],
 Brick #513: [(7, 3, 337), (8, 3, 337)] => [(7, 3) -> (8, 3), x], z=[337],
 Brick #188: [(8, 2, 336), (8, 4, 336)] => [(8, 2) -> (8, 4), y], z=[336],
 Brick #482: [(7, 5, 336), (9, 5, 336)] => [(7, 5) -> (9, 5), x], z=[336],
 Brick #778: [(4, 1, 336), (8, 1, 336)] => [(4, 1) -> (8, 1), x], z=[336],
 Brick #342: [(7, 1, 334), (7, 4, 334)] => [(7, 1) -> (7, 4), y], z=[334],
 Brick #1105: [(8, 3, 334), (8, 6, 334)] => [(8, 3) -> (8, 6), y], z=[334],
 Brick #407: [(6, 6, 332), (7, 6, 332)] => [(6, 6) -> (7, 6), x], z=[332],
 Brick #524: [(9, 2, 332), (9, 4, 332)] => [(9, 2) -> (9, 4), y], z=[332],
 Brick #848: [(6, 4, 331), (9, 4, 331)] => [(6, 4) -> (9, 4), x], z=[331],
 Brick #798

In [343]:
def draw_bricks(bricks, min_x=min_x, max_x=max_x, min_y=min_y, max_y=max_y):
    for y in range(max_y, min_y - 1, -1):
        row = ""
        for x in range(min_x, max_x + 1):
            point = XYPoint(x, y)
            found = False
            for brick in bricks:
                if brick.line.intersects(XYLineSegment(point, point)):
                    row += "*"
                    found = True
                    break
            if not found:
                row += "."
        print(row)

In [344]:
DEBUG = False

supported_by = defaultdict(list)
placed = []

# go from the bottom up; for each brick look at the ones below it
for brick_num in range(len(bricks) - 1, -1, -1):
    top_brick = bricks[brick_num]
    if DEBUG: print(f"Comparing {top_brick} to those below it.")

    new_bottom = None
    for below_brick in placed:
        if DEBUG: print(f"\tChecking {below_brick}...")
        touches = top_brick.line.intersects(below_brick.line)
        
        if touches:
            if DEBUG: print(f"\tTouches {below_brick}")
            top_brick_zs = top_brick.z
            below_brick_zs = below_brick.z

            if new_bottom is not None and below_brick_zs[-1] < (new_bottom - 1):
                if DEBUG: print(f"\t\tBut {below_brick} is below the new bottom {new_bottom}, so ignoring.")
                continue

            # move the top brick down to touch the below brick
            if DEBUG: print(f"\tMoving bottom of {top_brick.z} down to touch top of {below_brick.z}")
            delta = (top_brick_zs[0] - below_brick_zs[-1] - 1)
            top_brick.z = [z - delta for z in top_brick_zs]
            if DEBUG: print(f"\tNow at {top_brick.z}")
            supported_by[top_brick.id].append(below_brick.id)

            new_bottom = top_brick.z[0]
        
    # if it doesn't intersect anything below it, it can fall to the ground
    if new_bottom is None and top_brick.z[-1] >= 1:
        if DEBUG: print(f"\t{top_brick} can fall to the ground")
        top_brick_zs = top_brick.z
        delta = (top_brick_zs[0] - 1)
        top_brick.z = [z - delta for z in top_brick_zs]
        if DEBUG: print(f"\tNow at {top_brick.z}")
        supported_by[top_brick.id].append(None)

    placed.append(top_brick)
    placed = sorted(placed, key=lambda b: -b.z[-1])

    if DEBUG: draw_bricks(bricks[brick_num:])

In [345]:
bricks

[Brick #101: [(4, 5, 341), (7, 5, 341)] => [(4, 5) -> (7, 5), x], z=[167],
 Brick #43: [(7, 5, 339), (9, 5, 339)] => [(7, 5) -> (9, 5), x], z=[166],
 Brick #826: [(4, 1, 337), (4, 1, 339)] => [(4, 1) -> (4, 1), point], z=[166, 167, 168],
 Brick #889: [(6, 0, 338), (6, 1, 338)] => [(6, 0) -> (6, 1), y], z=[166],
 Brick #513: [(7, 3, 337), (8, 3, 337)] => [(7, 3) -> (8, 3), x], z=[166],
 Brick #188: [(8, 2, 336), (8, 4, 336)] => [(8, 2) -> (8, 4), y], z=[165],
 Brick #482: [(7, 5, 336), (9, 5, 336)] => [(7, 5) -> (9, 5), x], z=[165],
 Brick #778: [(4, 1, 336), (8, 1, 336)] => [(4, 1) -> (8, 1), x], z=[165],
 Brick #342: [(7, 1, 334), (7, 4, 334)] => [(7, 1) -> (7, 4), y], z=[164],
 Brick #1105: [(8, 3, 334), (8, 6, 334)] => [(8, 3) -> (8, 6), y], z=[164],
 Brick #407: [(6, 6, 332), (7, 6, 332)] => [(6, 6) -> (7, 6), x], z=[164],
 Brick #524: [(9, 2, 332), (9, 4, 332)] => [(9, 2) -> (9, 4), y], z=[164],
 Brick #848: [(6, 4, 331), (9, 4, 331)] => [(6, 4) -> (9, 4), x], z=[163],
 Brick #798

In [346]:
bricks_by_id = {b.id: b for b in bricks}

In [347]:
len(supported_by.keys())

1257

In [348]:
supports = defaultdict(list)
for k, vals in supported_by.items():
    for v in vals:
        if v is not None:
            #print(f"{bricks_by_id[v]} supports {bricks_by_id[k]}")
            # double-check that eveything is placed properly
            if (bricks_by_id[v].z[-1] + 1) != bricks_by_id[k].z[0]:
                print(f"Error: {bricks_by_id[v]} does not properly support {bricks_by_id[k]}")
            supports[v].append(k)

In [349]:
needed_bricks = set()
for k, vals in supported_by.items():
    print(f"{bricks_by_id[k]} is supported by bricks {[bricks_by_id[v] for v in vals if v is not None]}")
    # if a brick is the only support for another brick we can't demolish it
    if len(vals) == 1:
        if vals[0] is not None:
            needed_bricks.add(vals[0])

Brick #912: [(0, 0, 1), (2, 0, 1)] => [(0, 0) -> (2, 0), x], z=[1] is supported by bricks []
Brick #896: [(0, 9, 1), (2, 9, 1)] => [(0, 9) -> (2, 9), x], z=[1] is supported by bricks []
Brick #854: [(8, 8, 1), (9, 8, 1)] => [(8, 8) -> (9, 8), x], z=[1] is supported by bricks []
Brick #702: [(6, 3, 1), (6, 7, 1)] => [(6, 3) -> (6, 7), y], z=[1] is supported by bricks []
Brick #441: [(9, 3, 1), (9, 5, 1)] => [(9, 3) -> (9, 5), y], z=[1] is supported by bricks []
Brick #141: [(8, 9, 1), (8, 9, 1)] => [(8, 9) -> (8, 9), point], z=[1] is supported by bricks []
Brick #915: [(9, 1, 2), (9, 3, 2)] => [(9, 1) -> (9, 3), y], z=[2] is supported by bricks [Brick #441: [(9, 3, 1), (9, 5, 1)] => [(9, 3) -> (9, 5), y], z=[1]]
Brick #805: [(4, 1, 2), (5, 1, 2)] => [(4, 1) -> (5, 1), x], z=[1] is supported by bricks []
Brick #542: [(6, 8, 2), (7, 8, 2)] => [(6, 8) -> (7, 8), x], z=[1] is supported by bricks []
Brick #517: [(1, 6, 2), (4, 6, 2)] => [(1, 6) -> (4, 6), x], z=[1] is supported by bricks []


In [350]:
all_bricks = set([b.id for b in bricks])
len(all_bricks.difference(needed_bricks))

391

## Part 2

In [351]:
def get_removed_count(brick_id, supports=supports, supported_by=supported_by):
    queue = [brick_id]
    removed = set([brick_id])
    while len(queue) > 0:
        current = queue.pop(0)
        #print(f"Popped {test_ids_map[current]}")
        for next_brick in supports.get(current, []):
            next_supported_by = supported_by[next_brick]
            #print(f"\tNext brick {test_ids_map[next_brick]} is supported by {[test_ids_map[n] for n in next_supported_by]}")
            if all([s in removed for s in next_supported_by]):
                #print("\t\tCan remove it!")
                removed.add(next_brick)
                queue.append(next_brick)
            #print(f"Removed is {[test_ids_map[r] for r in removed]}")
    
    return len(removed) - 1

In [352]:
sum(get_removed_count(brick_id) for brick_id in all_bricks)

69601